# Julia companion notebook: merge time computation using DifferentialEquations.jl

In [1]:
using DifferentialEquations, DataFrames, CSV

In [2]:
df = CSV.read("df_julia_input.csv", DataFrame)

,Semimajor,Eccentricity,Mass_0,Mass_1,BWorldtime,GWtime
,Float64,Float64,Float64,Float64,Float64,Float64
1,8218.09,0.933191,43.8982,26.7488,6.25564,6.38255e9
2,134.886,0.00994416,23.0484,27.2323,4.97521,1.57202e6
3,481.943,0.0125616,36.26,32.6622,4.73317,9.90318e7
4,80356.4,0.0957917,38.3061,56.341,4.17184,2.96301e16
5,109.564,0.0144397,17.2293,17.3973,6.98481,2.08e6
6,206.199,0.0101597,21.0744,28.1394,5.00198,9.28317e6
7,159.284,0.0102363,24.0653,24.7807,5.47265,3.31179e6
8,8975.55,0.322961,25.9777,20.8143,6.63212,2.61538e13
9,79727.1,0.558309,53.849,46.3493,4.69829,6.55249e15


In [3]:
G = 3.925125598496094e+20
c = 13598865132357.05

1.359886513235705e13

## How does this work?
This notebook is NOT meant to carefully explain how to use DifferentialEquations.jl; for that there's a good amount of documentation on the internet. Moreover we won't really explain the details (as most are Julia specific), so here's a recap of what we need to do.          
In order to use DifferentialEquations.jl one needs to specify a problem i.e. an equation to solve, then solve it. It's possible to generalize this to an ensemble of e.g. different initial conditions/integration intervals/etc. (exactly what we need) by specifying a function that explains how to switch from a problem to the next one. Therefore this is what we're doing below:
- we create array containing the tuples of time intervals wrt integrate for every row, etc.;
- we define the base problem and how to "update" it in the ensemble;
- we solve the ensemble problem and save the solution.

In [4]:
# first of all we create a bunch of useful arrays
a_arr = df[!, "Semimajor"]

e_arr = df[!, "Eccentricity"]

u0_arr = [[a, e] for (a, e) in zip(a_arr, e_arr)]

t0_arr = df[!, "BWorldtime"]
GW_arr = df[!, "GWtime"]
T = 13.8e3

tspan_arr = [(t0, t1) for (t0, t1) in zip(t0_arr, GW_arr/2)]  # this /2 greatly improves speed without much impact on the result

M_arr = df[!, "Mass_0"]
m_arr = df[!, "Mass_1"]


b0 = -64*G^3/(5*c^5)
b1_arr = b0.*M_arr.*m_arr.*(M_arr+m_arr) # matlab-style elementwise product operator
b2 = 73/24
b3 = 37/96

d0 = -304*G^3/(15*c^5)
d1_arr = d0.*M_arr.*m_arr.*(M_arr+m_arr)
d2 = 121/304

k1 = 7/2
k2 = 5/2

# RHS of Peters' equation, inplace Julia friendly version
function f!(du, u, p, t)
    b0, b1, b2, b3, d0, d1, d2, k1, k2 = p
    du[1] = b1*(1+b2*u[2]^2+b3*u[2]^2)/(u[1]^3*(abs(1-u[2]^2)^k1))
    du[2] = d1*u[2]*(1+d2*u[2]^2)/(u[1]^4*(abs(1-u[2]^2)^k2))
end

f! (generic function with 1 method)

In [5]:
n = length(b1_arr)
x = [fill(b0, n), b1_arr, fill(b2, n), fill(b3, n), fill(d0, n), d1_arr, fill(d2, n), fill(k1, n), fill(k2, n)]
M = reduce(hcat, x)
params_arr = M

2500×9 Matrix{Float64}:
 -0.0016644  -138.071   3.04167  0.385417  …  -218.612   0.398026  3.5  2.5
 -0.0016644   -52.5273  3.04167  0.385417      -83.1682  0.398026  3.5  2.5
 -0.0016644  -135.859   3.04167  0.385417     -215.11    0.398026  3.5  2.5
 -0.0016644  -339.983   3.04167  0.385417     -538.306   0.398026  3.5  2.5
 -0.0016644   -17.275   3.04167  0.385417      -27.3521  0.398026  3.5  2.5
 -0.0016644   -48.5753  3.04167  0.385417  …   -76.9109  0.398026  3.5  2.5
 -0.0016644   -48.483   3.04167  0.385417      -76.7648  0.398026  3.5  2.5
 -0.0016644   -42.1106  3.04167  0.385417      -66.6751  0.398026  3.5  2.5
 -0.0016644  -416.235   3.04167  0.385417     -659.039   0.398026  3.5  2.5
 -0.0016644  -222.836   3.04167  0.385417     -352.823   0.398026  3.5  2.5
 -0.0016644  -200.804   3.04167  0.385417  …  -317.939   0.398026  3.5  2.5
 -0.0016644   -84.8978  3.04167  0.385417     -134.422   0.398026  3.5  2.5
 -0.0016644  -207.64    3.04167  0.385417     -328.763   0.39802

In [6]:
function prob_func(prob, i, repeat) # to update problems within the ensemble
  remake(prob, u0 = u0_arr[i], tspan = tspan_arr[i], p = params_arr[i,:]) 
end

output_func(sol,i) = (sol.t[end], false) # to tell Julia which part of the solution to save

output_func (generic function with 1 method)

In [7]:
# DE problem
prob = ODEProblem(f!, u0_arr[1], tspan_arr[1], params_arr[1,:]) 
# Ensemble of problems
probE = EnsembleProblem(prob, prob_func = prob_func, output_func = output_func)

# callback to stop the integration if a becomes negative (slightly different from the isco condition, but the result will of course be very close since the isco is always very small)
condition(u, t, integrator) = u[1] #< 0 # equivalently 
affect!(integrator) = terminate!(integrator)
#cb = DiscreteCallback(condition,affect!) # without interpolation
cb = ContinuousCallback(condition, affect!) # with interpolation

# we solve our ensemble with an automatic switch betwenn Tsit5 (an updated RK4) and Rosenbrock23 (an adaptive stiff solver similar to an updated LSODA)
@time sol = solve(probE, AutoTsit5(Rosenbrock32()), callback = cb, trajectories = size(params_arr)[1]; verbose = false)
# use ensemblealg = EnsembleDistributed() to easily parallelize on the CPU (not done here due to incomplete knowledge of Julia's Distributed library)

 37.006554 seconds (75.57 M allocations: 5.086 GiB, 3.21% gc time, 0.02% compilation time)


EnsembleSolution Solution of length 2500 with uType:
Float64

In [8]:
Vector(sol) # print the result

2500-element Vector{Float64}:
      3.191276e9
 786007.5
      4.95159e7
      1.4815035e16
      1.040001e6
      4.641584e6
      1.6558945e6
      1.307692e13
      3.276247e15
      4.708807e8
      2.914614e15
      2.7790115e11
      1.1501815e11
      ⋮
      7.33457e11
      1.2475935e6
      4.7793725e7
 414387.15
 283900.0
 718953.5
      4.856823e13
 480015.8
    136.12715
   1063.881
      0.6572720000000001
      2.8770005e10

In [9]:
CSV.write("df_julia_output.csv", (data = Vector(sol), )) # save the result

"df_julia_output.csv"